# Graphs and Graph Algorithms - Essential Patterns and Techniques

## Learning Objectives
- Master graph representations and traversal algorithms
- Understand DFS vs BFS decision making for graphs
- Practice shortest path and connectivity problems
- Learn cycle detection and topological sorting

## Key Patterns Covered
1. **Graph Representations**: Adjacency list, adjacency matrix
2. **Graph Traversal**: DFS and BFS implementations
3. **Shortest Path**: BFS for unweighted, Dijkstra for weighted
4. **Cycle Detection**: For directed and undirected graphs
5. **Topological Sorting**: Dependency resolution
6. **Connected Components**: Union-Find and graph traversal

---

## Graph Representation and Basic Operations

First, let's implement different ways to represent graphs and basic utility functions.

In [ ]:
from collections import defaultdict, deque
import heapq

class Graph:
    """
    Graph implementation using adjacency list.
    Supports both directed and undirected graphs.
    """
    
    def __init__(self, directed=False):
        self.graph = defaultdict(list)
        self.directed = directed
        self.vertices = set()
    
    def add_edge(self, u, v, weight=1):
        """Add edge from u to v with optional weight."""
        self.graph[u].append((v, weight))
        self.vertices.add(u)
        self.vertices.add(v)
        
        if not self.directed:
            self.graph[v].append((u, weight))
    
    def get_neighbors(self, vertex):
        """Get neighbors of a vertex."""
        return [(neighbor, weight) for neighbor, weight in self.graph[vertex]]
    
    def get_vertices(self):
        """Get all vertices in the graph."""
        return list(self.vertices)
    
    def display(self):
        """Display the graph."""
        for vertex in sorted(self.vertices):
            neighbors = [(n, w) for n, w in self.graph[vertex]]
            print(f"{vertex}: {neighbors}")

def create_adjacency_matrix(edges, num_vertices):
    """
    Create adjacency matrix from edge list.
    
    Args:
        edges: List of (u, v, weight) tuples
        num_vertices: Number of vertices
    
    Returns:
        2D matrix representing the graph
    """
    matrix = [[0] * num_vertices for _ in range(num_vertices)]
    
    for u, v, weight in edges:
        matrix[u][v] = weight
        # For undirected graph, uncomment the next line
        # matrix[v][u] = weight
    
    return matrix

def print_adjacency_matrix(matrix):
    """Print adjacency matrix in readable format."""
    n = len(matrix)
    print("   ", end="")
    for i in range(n):
        print(f"{i:3}", end="")
    print()
    
    for i in range(n):
        print(f"{i}: ", end="")
        for j in range(n):
            print(f"{matrix[i][j]:3}", end="")
        print()

# Test graph representations
print("=== Graph Representations Test ===")

# Create undirected graph
g = Graph(directed=False)
edges = [(0, 1), (0, 2), (1, 3), (2, 3), (3, 4)]

for u, v in edges:
    g.add_edge(u, v)

print("Undirected Graph (Adjacency List):")
g.display()

# Create adjacency matrix
print("\nAdjacency Matrix:")
matrix_edges = [(0, 1, 1), (0, 2, 1), (1, 3, 1), (2, 3, 1), (3, 4, 1)]
matrix = create_adjacency_matrix(matrix_edges, 5)
print_adjacency_matrix(matrix)

print("\nGraph vertices:", g.get_vertices())
print("Neighbors of vertex 0:", g.get_neighbors(0))

## Problem 1: Graph Traversal - DFS and BFS

**Problem**: Implement depth-first search (DFS) and breadth-first search (BFS) for graph traversal.

**Approaches**: 
- **DFS**: Use stack (iterative) or recursion, goes deep first
- **BFS**: Use queue, explores neighbors level by level
- Track visited nodes to avoid infinite loops

**Time Complexity**: O(V + E) | **Space Complexity**: O(V)

In [ ]:
def dfs_recursive(graph, start, visited=None):
    """
    Depth-First Search using recursion.
    
    Args:
        graph: Graph object
        start: Starting vertex
        visited: Set of visited vertices
    
    Returns:
        List of vertices in DFS order
    """
    if visited is None:
        visited = set()
    
    visited.add(start)
    result = [start]
    
    # Visit all unvisited neighbors
    for neighbor, _ in graph.get_neighbors(start):
        if neighbor not in visited:
            result.extend(dfs_recursive(graph, neighbor, visited))
    
    return result

def dfs_iterative(graph, start):
    """
    Depth-First Search using iterative approach with stack.
    """
    visited = set()
    stack = [start]
    result = []
    
    while stack:
        vertex = stack.pop()
        
        if vertex not in visited:
            visited.add(vertex)
            result.append(vertex)
            
            # Add neighbors to stack (reverse order for consistent ordering)
            neighbors = [n for n, _ in graph.get_neighbors(vertex)]
            for neighbor in reversed(neighbors):
                if neighbor not in visited:
                    stack.append(neighbor)
    
    return result

def bfs(graph, start):
    """
    Breadth-First Search using queue.
    
    Args:
        graph: Graph object
        start: Starting vertex
    
    Returns:
        List of vertices in BFS order
    """
    visited = set()
    queue = deque([start])
    result = []
    
    while queue:
        vertex = queue.popleft()
        
        if vertex not in visited:
            visited.add(vertex)
            result.append(vertex)
            
            # Add neighbors to queue
            for neighbor, _ in graph.get_neighbors(vertex):
                if neighbor not in visited:
                    queue.append(neighbor)
    
    return result

def bfs_level_order(graph, start):
    """
    BFS that returns vertices grouped by level/distance.
    """
    visited = set()
    queue = deque([(start, 0)])  # (vertex, level)
    levels = defaultdict(list)
    
    while queue:
        vertex, level = queue.popleft()
        
        if vertex not in visited:
            visited.add(vertex)
            levels[level].append(vertex)
            
            # Add neighbors at next level
            for neighbor, _ in graph.get_neighbors(vertex):
                if neighbor not in visited:
                    queue.append((neighbor, level + 1))
    
    return dict(levels)

def connected_components(graph):
    """
    Find all connected components in undirected graph.
    """
    visited = set()
    components = []
    
    for vertex in graph.get_vertices():
        if vertex not in visited:
            # Start DFS from this vertex
            component = []
            stack = [vertex]
            
            while stack:
                v = stack.pop()
                if v not in visited:
                    visited.add(v)
                    component.append(v)
                    
                    for neighbor, _ in graph.get_neighbors(v):
                        if neighbor not in visited:
                            stack.append(neighbor)
            
            components.append(sorted(component))
    
    return components

# Test graph traversal algorithms
print("=== Graph Traversal Test ===")

# Create test graph
g = Graph(directed=False)
edges = [(0, 1), (0, 2), (1, 3), (2, 4), (3, 5), (4, 5)]

for u, v in edges:
    g.add_edge(u, v)

print("Graph structure:")
g.display()
print()

start_vertex = 0
print(f"Starting traversal from vertex {start_vertex}:")

# Test DFS
dfs_rec_result = dfs_recursive(g, start_vertex)
dfs_iter_result = dfs_iterative(g, start_vertex)
print(f"DFS (recursive): {dfs_rec_result}")
print(f"DFS (iterative): {dfs_iter_result}")

# Test BFS
bfs_result = bfs(g, start_vertex)
bfs_levels = bfs_level_order(g, start_vertex)
print(f"BFS: {bfs_result}")
print(f"BFS by levels: {dict(sorted(bfs_levels.items()))}")

# Test connected components
print(f"Connected components: {connected_components(g)}")

# Test with disconnected graph
print("\n=== Disconnected Graph Test ===")
g2 = Graph(directed=False)
disconnected_edges = [(0, 1), (0, 2), (3, 4), (5, 6)]

for u, v in disconnected_edges:
    g2.add_edge(u, v)

print("Disconnected graph structure:")
g2.display()
print(f"Connected components: {connected_components(g2)}")

## Problem 2: Shortest Path in Unweighted Graph

**Problem**: Find shortest path between two vertices in unweighted graph.

**Approach**: BFS guarantees shortest path in unweighted graphs
- Use BFS to explore vertices level by level
- Track parent pointers to reconstruct path
- Distance = level in BFS tree

**Time Complexity**: O(V + E) | **Space Complexity**: O(V)

In [ ]:
def shortest_path_unweighted(graph, start, end):
    """
    Find shortest path in unweighted graph using BFS.
    
    Args:
        graph: Graph object
        start: Starting vertex
        end: Target vertex
    
    Returns:
        Tuple: (distance, path) or (None, None) if no path exists
    """
    if start == end:
        return (0, [start])
    
    visited = set()
    queue = deque([(start, 0)])  # (vertex, distance)
    parent = {start: None}
    
    while queue:
        vertex, distance = queue.popleft()
        
        if vertex in visited:
            continue
            
        visited.add(vertex)
        
        # Check if we reached the target
        if vertex == end:
            # Reconstruct path
            path = []
            current = end
            
            while current is not None:
                path.append(current)
                current = parent[current]
            
            path.reverse()
            return (distance, path)
        
        # Explore neighbors
        for neighbor, _ in graph.get_neighbors(vertex):
            if neighbor not in visited and neighbor not in parent:
                parent[neighbor] = vertex
                queue.append((neighbor, distance + 1))
    
    return (None, None)  # No path found

def all_shortest_paths_bfs(graph, start):
    """
    Find shortest paths from start vertex to all other vertices.
    
    Returns:
        Dictionary: vertex -> (distance, path)
    """
    distances = {start: 0}
    parent = {start: None}
    queue = deque([start])
    visited = set()
    
    while queue:
        vertex = queue.popleft()
        
        if vertex in visited:
            continue
            
        visited.add(vertex)
        
        for neighbor, _ in graph.get_neighbors(vertex):
            if neighbor not in distances:
                distances[neighbor] = distances[vertex] + 1
                parent[neighbor] = vertex
                queue.append(neighbor)
    
    # Build result with paths
    result = {}
    for vertex in distances:
        # Reconstruct path
        path = []
        current = vertex
        
        while current is not None:
            path.append(current)
            current = parent[current]
        
        path.reverse()
        result[vertex] = (distances[vertex], path)
    
    return result

# Test shortest path algorithms
print("=== Shortest Path Test ===")

# Create test graph
g = Graph(directed=False)
edges = [(0, 1), (0, 2), (1, 3), (2, 3), (3, 4), (2, 4)]

for u, v in edges:
    g.add_edge(u, v)

print("Graph structure:")
g.display()
print()

# Test shortest path between specific vertices
test_pairs = [(0, 4), (0, 3), (1, 2), (0, 5)]  # Last one has no path

for start, end in test_pairs:
    distance, path = shortest_path_unweighted(g, start, end)
    if distance is not None:
        print(f"Shortest path from {start} to {end}: distance = {distance}, path = {path}")
    else:
        print(f"No path from {start} to {end}")

# Test all shortest paths from vertex 0
print(f"\nAll shortest paths from vertex 0:")
all_paths = all_shortest_paths_bfs(g, 0)
for vertex, (distance, path) in sorted(all_paths.items()):
    print(f"To {vertex}: distance = {distance}, path = {path}")

## Problem 3: Dijkstra's Algorithm for Weighted Graphs

**Problem**: Find shortest path in weighted graph with non-negative edge weights.

**Approach**: Greedy algorithm using priority queue (min-heap)
- Always process vertex with minimum distance first
- Update distances to neighbors if shorter path found
- Use priority queue to efficiently get minimum distance vertex

**Time Complexity**: O((V + E) log V) | **Space Complexity**: O(V)

In [ ]:
def dijkstra(graph, start):
    """
    Dijkstra's algorithm for shortest paths in weighted graph.
    
    Args:
        graph: Graph object with weighted edges
        start: Starting vertex
    
    Returns:
        Tuple: (distances dict, parent dict)
    """
    # Initialize distances and parent pointers
    distances = {vertex: float('inf') for vertex in graph.get_vertices()}
    distances[start] = 0
    parent = {vertex: None for vertex in graph.get_vertices()}
    
    # Priority queue: (distance, vertex)
    pq = [(0, start)]
    visited = set()
    
    while pq:
        current_distance, current_vertex = heapq.heappop(pq)
        
        # Skip if already visited
        if current_vertex in visited:
            continue
        
        visited.add(current_vertex)
        
        # Update distances to neighbors
        for neighbor, weight in graph.get_neighbors(current_vertex):
            if neighbor not in visited:
                new_distance = distances[current_vertex] + weight
                
                if new_distance < distances[neighbor]:
                    distances[neighbor] = new_distance
                    parent[neighbor] = current_vertex
                    heapq.heappush(pq, (new_distance, neighbor))
    
    return distances, parent

def get_shortest_path_dijkstra(parent, start, end):
    """
    Reconstruct shortest path from Dijkstra's parent pointers.
    """
    if parent[end] is None and start != end:
        return None  # No path exists
    
    path = []
    current = end
    
    while current is not None:
        path.append(current)
        current = parent[current]
    
    path.reverse()
    return path

def dijkstra_single_target(graph, start, end):
    """
    Optimized Dijkstra for single target (stops when target is reached).
    """
    distances = {vertex: float('inf') for vertex in graph.get_vertices()}
    distances[start] = 0
    parent = {vertex: None for vertex in graph.get_vertices()}
    
    pq = [(0, start)]
    visited = set()
    
    while pq:
        current_distance, current_vertex = heapq.heappop(pq)
        
        if current_vertex in visited:
            continue
        
        visited.add(current_vertex)
        
        # Early termination if we reached the target
        if current_vertex == end:
            path = get_shortest_path_dijkstra(parent, start, end)
            return distances[end], path
        
        # Update neighbors
        for neighbor, weight in graph.get_neighbors(current_vertex):
            if neighbor not in visited:
                new_distance = distances[current_vertex] + weight
                
                if new_distance < distances[neighbor]:
                    distances[neighbor] = new_distance
                    parent[neighbor] = current_vertex
                    heapq.heappush(pq, (new_distance, neighbor))
    
    return float('inf'), None  # No path found

# Test Dijkstra's algorithm
print("=== Dijkstra's Algorithm Test ===")

# Create weighted graph
g = Graph(directed=False)
weighted_edges = [
    (0, 1, 4), (0, 2, 2), (1, 2, 1), (1, 3, 5),
    (2, 3, 8), (2, 4, 10), (3, 4, 2), (3, 5, 6), (4, 5, 3)
]

for u, v, w in weighted_edges:
    g.add_edge(u, v, w)

print("Weighted graph structure:")
g.display()
print()

# Run Dijkstra from vertex 0
start_vertex = 0
distances, parent = dijkstra(g, start_vertex)

print(f"Shortest distances from vertex {start_vertex}:")
for vertex in sorted(distances.keys()):
    distance = distances[vertex]
    if distance == float('inf'):
        print(f"To {vertex}: No path")
    else:
        path = get_shortest_path_dijkstra(parent, start_vertex, vertex)
        print(f"To {vertex}: distance = {distance}, path = {path}")

# Test single target optimization
print(f"\nSingle target test (0 to 5):")
distance, path = dijkstra_single_target(g, 0, 5)
print(f"Distance: {distance}, Path: {path}")

# Compare with different starting points
print(f"\nDifferent starting points to vertex 5:")
for start in [0, 1, 2, 3, 4]:
    distance, path = dijkstra_single_target(g, start, 5)
    print(f"From {start}: distance = {distance}, path = {path}")

## Problem 4: Cycle Detection in Graphs

**Problem**: Detect if a graph contains a cycle.

**Approaches**: Different methods for directed vs undirected graphs
- **Undirected**: DFS with parent tracking or Union-Find
- **Directed**: DFS with color coding (white/gray/black)
- **Gray nodes**: Currently being processed (in recursion stack)

**Time Complexity**: O(V + E) | **Space Complexity**: O(V)

In [ ]:
def has_cycle_undirected_dfs(graph):
    """
    Detect cycle in undirected graph using DFS.
    
    Args:
        graph: Undirected graph object
    
    Returns:
        Boolean indicating presence of cycle
    """
    visited = set()
    
    def dfs(vertex, parent):
        visited.add(vertex)
        
        for neighbor, _ in graph.get_neighbors(vertex):
            if neighbor not in visited:
                if dfs(neighbor, vertex):
                    return True
            elif parent != neighbor:  # Back edge found (not to parent)
                return True
        
        return False
    
    # Check each component
    for vertex in graph.get_vertices():
        if vertex not in visited:
            if dfs(vertex, -1):
                return True
    
    return False

def has_cycle_directed_dfs(graph):
    """
    Detect cycle in directed graph using DFS with color coding.
    
    Colors:
    - White (0): Unvisited
    - Gray (1): Visiting (in current DFS path)
    - Black (2): Visited (completely processed)
    """
    WHITE, GRAY, BLACK = 0, 1, 2
    colors = {vertex: WHITE for vertex in graph.get_vertices()}
    
    def dfs(vertex):
        if colors[vertex] == GRAY:  # Back edge to ancestor
            return True
        if colors[vertex] == BLACK:  # Already processed
            return False
        
        colors[vertex] = GRAY  # Mark as visiting
        
        # Visit all neighbors
        for neighbor, _ in graph.get_neighbors(vertex):
            if dfs(neighbor):
                return True
        
        colors[vertex] = BLACK  # Mark as completely processed
        return False
    
    # Check each vertex
    for vertex in graph.get_vertices():
        if colors[vertex] == WHITE:
            if dfs(vertex):
                return True
    
    return False

def find_cycle_path_directed(graph):
    """
    Find actual cycle path in directed graph.
    
    Returns:
        List representing cycle path, or None if no cycle
    """
    WHITE, GRAY, BLACK = 0, 1, 2
    colors = {vertex: WHITE for vertex in graph.get_vertices()}
    parent = {}
    cycle_start = None
    
    def dfs(vertex):
        nonlocal cycle_start
        
        if colors[vertex] == GRAY:
            cycle_start = vertex
            return True
        if colors[vertex] == BLACK:
            return False
        
        colors[vertex] = GRAY
        
        for neighbor, _ in graph.get_neighbors(vertex):
            parent[neighbor] = vertex
            if dfs(neighbor):
                return True
        
        colors[vertex] = BLACK
        return False
    
    # Find cycle
    for vertex in graph.get_vertices():
        if colors[vertex] == WHITE:
            if dfs(vertex):
                break
    
    if cycle_start is None:
        return None
    
    # Reconstruct cycle
    cycle = []
    current = cycle_start
    
    while True:
        cycle.append(current)
        current = parent[current]
        if current == cycle_start:
            break
    
    cycle.append(cycle_start)
    cycle.reverse()
    return cycle

class UnionFind:
    """
    Union-Find data structure for cycle detection.
    """
    
    def __init__(self, vertices):
        self.parent = {v: v for v in vertices}
        self.rank = {v: 0 for v in vertices}
    
    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])  # Path compression
        return self.parent[x]
    
    def union(self, x, y):
        root_x = self.find(x)
        root_y = self.find(y)
        
        if root_x == root_y:
            return False  # Cycle detected
        
        # Union by rank
        if self.rank[root_x] < self.rank[root_y]:
            self.parent[root_x] = root_y
        elif self.rank[root_x] > self.rank[root_y]:
            self.parent[root_y] = root_x
        else:
            self.parent[root_y] = root_x
            self.rank[root_x] += 1
        
        return True

def has_cycle_undirected_union_find(graph):
    """
    Detect cycle in undirected graph using Union-Find.
    """
    uf = UnionFind(graph.get_vertices())
    processed_edges = set()
    
    for vertex in graph.get_vertices():
        for neighbor, _ in graph.get_neighbors(vertex):
            # Avoid processing same edge twice in undirected graph
            edge = tuple(sorted([vertex, neighbor]))
            if edge not in processed_edges:
                processed_edges.add(edge)
                if not uf.union(vertex, neighbor):
                    return True
    
    return False

# Test cycle detection
print("=== Cycle Detection Test ===")

# Test undirected graph without cycle
g1 = Graph(directed=False)
edges1 = [(0, 1), (1, 2), (2, 3)]
for u, v in edges1:
    g1.add_edge(u, v)

print("Undirected graph without cycle:")
g1.display()
print(f"Has cycle (DFS): {has_cycle_undirected_dfs(g1)}")
print(f"Has cycle (Union-Find): {has_cycle_undirected_union_find(g1)}")

# Test undirected graph with cycle
g2 = Graph(directed=False)
edges2 = [(0, 1), (1, 2), (2, 3), (3, 0)]
for u, v in edges2:
    g2.add_edge(u, v)

print("\nUndirected graph with cycle:")
g2.display()
print(f"Has cycle (DFS): {has_cycle_undirected_dfs(g2)}")
print(f"Has cycle (Union-Find): {has_cycle_undirected_union_find(g2)}")

# Test directed graph with cycle
g3 = Graph(directed=True)
edges3 = [(0, 1), (1, 2), (2, 3), (3, 1)]  # Cycle: 1 -> 2 -> 3 -> 1
for u, v in edges3:
    g3.add_edge(u, v)

print("\nDirected graph with cycle:")
g3.display()
print(f"Has cycle: {has_cycle_directed_dfs(g3)}")
cycle_path = find_cycle_path_directed(g3)
print(f"Cycle path: {cycle_path}")

# Test directed graph without cycle (DAG)
g4 = Graph(directed=True)
edges4 = [(0, 1), (1, 2), (0, 3), (3, 2)]
for u, v in edges4:
    g4.add_edge(u, v)

print("\nDirected acyclic graph (DAG):")
g4.display()
print(f"Has cycle: {has_cycle_directed_dfs(g4)}")
cycle_path = find_cycle_path_directed(g4)
print(f"Cycle path: {cycle_path}")

## Problem 5: Topological Sorting

**Problem**: Find a linear ordering of vertices in a directed acyclic graph (DAG) such that for every directed edge (u,v), vertex u comes before v.

**Approaches**: 
- **Kahn's Algorithm**: Use in-degree counting with queue (BFS-based)
- **DFS-based**: Use DFS and reverse postorder traversal
- Only works on DAGs (no cycles)

**Time Complexity**: O(V + E) | **Space Complexity**: O(V)

In [ ]:
def topological_sort_kahn(graph):
    """
    Topological sorting using Kahn's algorithm (BFS-based).
    
    Args:
        graph: Directed graph object
    
    Returns:
        List of vertices in topological order, or None if cycle exists
    """
    # Calculate in-degrees
    in_degree = {vertex: 0 for vertex in graph.get_vertices()}
    
    for vertex in graph.get_vertices():
        for neighbor, _ in graph.get_neighbors(vertex):
            in_degree[neighbor] += 1
    
    # Initialize queue with vertices having in-degree 0
    queue = deque([vertex for vertex in in_degree if in_degree[vertex] == 0])
    result = []
    
    while queue:
        vertex = queue.popleft()
        result.append(vertex)
        
        # Update in-degrees of neighbors
        for neighbor, _ in graph.get_neighbors(vertex):
            in_degree[neighbor] -= 1
            if in_degree[neighbor] == 0:
                queue.append(neighbor)
    
    # Check if all vertices are processed (no cycle)
    if len(result) != len(graph.get_vertices()):
        return None  # Cycle detected
    
    return result

def topological_sort_dfs(graph):
    """
    Topological sorting using DFS (reverse postorder).
    
    Returns:
        List of vertices in topological order, or None if cycle exists
    """
    WHITE, GRAY, BLACK = 0, 1, 2
    colors = {vertex: WHITE for vertex in graph.get_vertices()}
    result = []
    has_cycle = False
    
    def dfs(vertex):
        nonlocal has_cycle
        
        if colors[vertex] == GRAY:  # Back edge (cycle)
            has_cycle = True
            return
        if colors[vertex] == BLACK:
            return
        
        colors[vertex] = GRAY
        
        for neighbor, _ in graph.get_neighbors(vertex):
            dfs(neighbor)
            if has_cycle:
                return
        
        colors[vertex] = BLACK
        result.append(vertex)  # Add to result after processing all children
    
    # Process all vertices
    for vertex in graph.get_vertices():
        if colors[vertex] == WHITE:
            dfs(vertex)
            if has_cycle:
                return None
    
    return result[::-1]  # Reverse to get correct topological order

def all_topological_sorts(graph):
    """
    Find all possible topological sorts of a DAG.
    
    Returns:
        List of all possible topological orderings
    """
    def backtrack(current_order, remaining_vertices, in_degree):
        if not remaining_vertices:
            all_sorts.append(current_order[:])
            return
        
        # Try all vertices with in-degree 0
        for vertex in remaining_vertices[:]:
            if in_degree[vertex] == 0:
                # Choose this vertex
                current_order.append(vertex)
                remaining_vertices.remove(vertex)
                
                # Update in-degrees of neighbors
                decremented = []
                for neighbor, _ in graph.get_neighbors(vertex):
                    in_degree[neighbor] -= 1
                    decremented.append(neighbor)
                
                # Recurse
                backtrack(current_order, remaining_vertices, in_degree)
                
                # Backtrack: restore state
                for neighbor in decremented:
                    in_degree[neighbor] += 1
                remaining_vertices.append(vertex)
                current_order.pop()
    
    # Calculate in-degrees
    in_degree = {vertex: 0 for vertex in graph.get_vertices()}
    for vertex in graph.get_vertices():
        for neighbor, _ in graph.get_neighbors(vertex):
            in_degree[neighbor] += 1
    
    all_sorts = []
    remaining = graph.get_vertices()[:]
    backtrack([], remaining, in_degree)
    
    return all_sorts

# Test topological sorting
print("=== Topological Sorting Test ===")

# Create DAG for course prerequisites
# Courses: 0=Math, 1=Physics, 2=Chemistry, 3=Biology, 4=Advanced Physics
course_graph = Graph(directed=True)
prerequisites = [
    (0, 1),  # Math -> Physics
    (0, 2),  # Math -> Chemistry
    (1, 4),  # Physics -> Advanced Physics
    (2, 3),  # Chemistry -> Biology
]

for u, v in prerequisites:
    course_graph.add_edge(u, v)

print("Course prerequisite graph (DAG):")
course_graph.display()

# Test both algorithms
topo_kahn = topological_sort_kahn(course_graph)
topo_dfs = topological_sort_dfs(course_graph)

print(f"\nTopological order (Kahn's): {topo_kahn}")
print(f"Topological order (DFS): {topo_dfs}")

# Verify the ordering
def verify_topological_order(graph, order):
    """Verify if the given order is a valid topological ordering."""
    if not order:
        return False
    
    position = {vertex: i for i, vertex in enumerate(order)}
    
    for vertex in graph.get_vertices():
        for neighbor, _ in graph.get_neighbors(vertex):
            if position[vertex] >= position[neighbor]:
                return False
    
    return True

print(f"Kahn's result is valid: {verify_topological_order(course_graph, topo_kahn)}")
print(f"DFS result is valid: {verify_topological_order(course_graph, topo_dfs)}")

# Test with graph containing cycle
cyclic_graph = Graph(directed=True)
cyclic_edges = [(0, 1), (1, 2), (2, 0)]  # Creates a cycle
for u, v in cyclic_edges:
    cyclic_graph.add_edge(u, v)

print("\nGraph with cycle:")
cyclic_graph.display()
topo_cyclic = topological_sort_kahn(cyclic_graph)
print(f"Topological sort result: {topo_cyclic}")

# Find all possible topological sorts for small DAG
small_dag = Graph(directed=True)
small_edges = [(0, 2), (1, 2)]
for u, v in small_edges:
    small_dag.add_edge(u, v)

print("\nSmall DAG for all topological sorts:")
small_dag.display()
all_sorts = all_topological_sorts(small_dag)
print(f"All possible topological sorts: {all_sorts}")

## Problem 6: Graph Coloring and Bipartite Check

**Problem**: Determine if a graph is bipartite (2-colorable) and implement graph coloring.

**Approach**: Use BFS or DFS to attempt 2-coloring
- Start with any vertex, color it with color 0
- Color all neighbors with opposite color (1)
- If we encounter a neighbor with same color, graph is not bipartite

**Time Complexity**: O(V + E) | **Space Complexity**: O(V)

In [ ]:
def is_bipartite_bfs(graph):
    """
    Check if graph is bipartite using BFS.
    
    Args:
        graph: Undirected graph object
    
    Returns:
        Tuple: (is_bipartite, coloring_dict)
    """
    colors = {}
    
    def bfs_color(start):
        queue = deque([start])
        colors[start] = 0  # Start with color 0
        
        while queue:
            vertex = queue.popleft()
            current_color = colors[vertex]
            
            for neighbor, _ in graph.get_neighbors(vertex):
                if neighbor not in colors:
                    # Color neighbor with opposite color
                    colors[neighbor] = 1 - current_color
                    queue.append(neighbor)
                elif colors[neighbor] == current_color:
                    # Same color as current vertex - not bipartite
                    return False
        
        return True
    
    # Check each connected component
    for vertex in graph.get_vertices():
        if vertex not in colors:
            if not bfs_color(vertex):
                return False, {}
    
    return True, colors

def is_bipartite_dfs(graph):
    """
    Check if graph is bipartite using DFS.
    """
    colors = {}
    
    def dfs_color(vertex, color):
        colors[vertex] = color
        
        for neighbor, _ in graph.get_neighbors(vertex):
            if neighbor not in colors:
                if not dfs_color(neighbor, 1 - color):
                    return False
            elif colors[neighbor] == color:
                return False
        
        return True
    
    # Check each connected component
    for vertex in graph.get_vertices():
        if vertex not in colors:
            if not dfs_color(vertex, 0):
                return False, {}
    
    return True, colors

def graph_coloring_greedy(graph, num_colors):
    """
    Graph coloring using greedy algorithm.
    
    Args:
        graph: Graph object
        num_colors: Maximum number of colors to use
    
    Returns:
        Dictionary mapping vertices to colors, or None if impossible
    """
    coloring = {}
    vertices = sorted(graph.get_vertices())
    
    for vertex in vertices:
        # Find colors used by neighbors
        neighbor_colors = set()
        for neighbor, _ in graph.get_neighbors(vertex):
            if neighbor in coloring:
                neighbor_colors.add(coloring[neighbor])
        
        # Find first available color
        for color in range(num_colors):
            if color not in neighbor_colors:
                coloring[vertex] = color
                break
        else:
            # No available color found
            return None
    
    return coloring

def chromatic_number_upper_bound(graph):
    """
    Calculate upper bound for chromatic number using degree.
    Brooks' theorem: chromatic number <= max_degree + 1
    """
    if not graph.get_vertices():
        return 0
    
    max_degree = 0
    for vertex in graph.get_vertices():
        degree = len(graph.get_neighbors(vertex))
        max_degree = max(max_degree, degree)
    
    return max_degree + 1

def find_minimum_coloring(graph):
    """
    Find minimum number of colors needed to color the graph.
    """
    # First check if bipartite (2-colorable)
    is_bip, _ = is_bipartite_bfs(graph)
    if is_bip and graph.get_vertices():  # Non-empty bipartite graph
        return 2
    
    # Try increasing number of colors
    upper_bound = chromatic_number_upper_bound(graph)
    
    for num_colors in range(1, upper_bound + 1):
        coloring = graph_coloring_greedy(graph, num_colors)
        if coloring is not None:
            return num_colors, coloring
    
    return upper_bound, None

# Test bipartite and graph coloring
print("=== Bipartite and Graph Coloring Test ===")

# Test bipartite graph (even cycle)
bipartite_graph = Graph(directed=False)
bipartite_edges = [(0, 1), (1, 2), (2, 3), (3, 0)]  # Square
for u, v in bipartite_edges:
    bipartite_graph.add_edge(u, v)

print("Bipartite graph (square):")
bipartite_graph.display()
is_bip_bfs, colors_bfs = is_bipartite_bfs(bipartite_graph)
is_bip_dfs, colors_dfs = is_bipartite_dfs(bipartite_graph)
print(f"Is bipartite (BFS): {is_bip_bfs}, colors: {colors_bfs}")
print(f"Is bipartite (DFS): {is_bip_dfs}, colors: {colors_dfs}")

# Test non-bipartite graph (triangle)
triangle_graph = Graph(directed=False)
triangle_edges = [(0, 1), (1, 2), (2, 0)]
for u, v in triangle_edges:
    triangle_graph.add_edge(u, v)

print("\nNon-bipartite graph (triangle):")
triangle_graph.display()
is_bip, colors = is_bipartite_bfs(triangle_graph)
print(f"Is bipartite: {is_bip}, colors: {colors}")

# Test graph coloring
print(f"\nGraph coloring for triangle:")
min_colors, coloring = find_minimum_coloring(triangle_graph)
print(f"Minimum colors needed: {min_colors}")
print(f"Coloring: {coloring}")

# Test larger graph
larger_graph = Graph(directed=False)
larger_edges = [(0, 1), (0, 2), (1, 3), (2, 3), (3, 4), (4, 5), (5, 3)]
for u, v in larger_edges:
    larger_graph.add_edge(u, v)

print("\nLarger graph:")
larger_graph.display()
upper_bound = chromatic_number_upper_bound(larger_graph)
print(f"Upper bound for chromatic number: {upper_bound}")

min_colors, coloring = find_minimum_coloring(larger_graph)
print(f"Minimum colors needed: {min_colors}")
print(f"Coloring: {coloring}")

# Verify coloring is valid
def verify_coloring(graph, coloring):
    """Verify that no two adjacent vertices have the same color."""
    for vertex in graph.get_vertices():
        for neighbor, _ in graph.get_neighbors(vertex):
            if coloring[vertex] == coloring[neighbor]:
                return False
    return True

if coloring:
    is_valid = verify_coloring(larger_graph, coloring)
    print(f"Coloring is valid: {is_valid}")

## Summary and Key Takeaways

### Graph Algorithm Decision Making:

#### **When to Use DFS vs BFS:**
1. **Use DFS when**:
   - Finding all paths or exploring deeply
   - Cycle detection in directed graphs
   - Topological sorting
   - Memory is limited (O(h) vs O(w) space)

2. **Use BFS when**:
   - Finding shortest path in unweighted graphs
   - Level-by-level processing needed
   - Minimum steps problems
   - Bipartite checking

#### **Graph Representation Choice:**
- **Adjacency List**: Better for sparse graphs, easier neighbor iteration
- **Adjacency Matrix**: Better for dense graphs, O(1) edge lookup

### Essential Graph Patterns:

#### **1. Graph Traversal Template:**
```python
def graph_traversal(graph, start):
    visited = set()
    # Use stack for DFS, queue for BFS
    container = [start]  # or deque([start])
    
    while container:
        vertex = container.pop()  # or popleft() for BFS
        if vertex in visited:
            continue
        
        visited.add(vertex)
        # Process vertex
        
        for neighbor in graph.neighbors(vertex):
            if neighbor not in visited:
                container.append(neighbor)
```

#### **2. Shortest Path Selection:**
- **Unweighted graphs**: BFS (O(V + E))
- **Weighted graphs (non-negative)**: Dijkstra (O((V + E) log V))
- **Weighted graphs (negative edges)**: Bellman-Ford (O(VE))
- **All pairs**: Floyd-Warshall (O(V³))

#### **3. Cycle Detection Strategy:**
- **Undirected**: DFS with parent tracking or Union-Find
- **Directed**: DFS with color coding (white/gray/black)

#### **4. Topological Sort Applications:**
- Course scheduling with prerequisites
- Task dependency resolution
- Build systems and compilation order

### Time/Space Complexity Summary:
| Algorithm | Time | Space | Use Case |
|-----------|------|-------|----------|
| DFS/BFS | O(V + E) | O(V) | Traversal, connectivity |
| Dijkstra | O((V + E) log V) | O(V) | Weighted shortest path |
| Topological Sort | O(V + E) | O(V) | Ordering in DAG |
| Union-Find | O(α(n)) per op | O(V) | Cycle detection, connectivity |
| Graph Coloring | O(V + E) | O(V) | Bipartite check |

### Common Graph Problems Patterns:
1. **Connectivity**: DFS/BFS, Union-Find
2. **Shortest Path**: BFS (unweighted), Dijkstra (weighted)
3. **Cycle Detection**: DFS with state tracking
4. **Dependency Resolution**: Topological sorting
5. **Bipartite Matching**: 2-coloring with BFS/DFS
6. **Minimum Spanning Tree**: Kruskal's or Prim's (not covered here)

### Interview Tips:
1. **Clarify graph properties**: Directed/undirected, weighted/unweighted, dense/sparse
2. **Choose representation**: Adjacency list for most problems
3. **Handle disconnected graphs**: Process all components
4. **Track visited nodes**: Essential to avoid infinite loops
5. **Consider space constraints**: DFS uses less space than BFS

### Key Concepts Mastered:
- ✅ Graph representations and basic operations
- ✅ DFS and BFS traversal algorithms
- ✅ Shortest path algorithms (BFS, Dijkstra)
- ✅ Cycle detection for directed and undirected graphs
- ✅ Topological sorting with multiple approaches
- ✅ Bipartite graphs and graph coloring
- ✅ Connected components and Union-Find

---

**Next Steps**: Practice these algorithms on various graph problems, focusing on identifying the right algorithm and approach based on problem constraints!